## Chapter 2-3, 2-2강 시계열 예측 모델링 — PyTorch LSTM (Bike Sharing)

- **목표**: 시계열을 시퀀스 형태로 변환하고 LSTM으로 예측 학습/평가합니다.
- **데이터**: Kaggle Bike Sharing Demand (시간 단위, `count` 대상)
- **규칙(강의용)**: `matplotlib`만 사용 (seaborn X), 색상 지정 X, 서브플롯 X


### 0. 환경 준비 및 라이브러리 임포트
- PyTorch LSTM 구현 (GPU 없어도 CPU에서 동작)
- 재현성을 위한 시드 고정


In [ ]:
# -*- coding: utf-8 -*-
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

def _set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)



### 1. 데이터 준비 및 전처리
- `bike-sharing-demand/train.csv` 로드, 시간 파생변수 생성
- train/val/test 시간 분할


In [ ]:
def load_hourly_data():
    path = 'bike-sharing-demand/train.csv'
    if not os.path.exists(path):
        raise FileNotFoundError('train.csv 경로를 찾을 수 없습니다.')
    df = pd.read_csv(path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime').reset_index(drop=True)
    return df


def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['year'] = out['datetime'].dt.year
    out['month'] = out['datetime'].dt.month
    out['day'] = out['datetime'].dt.day
    out['dayofweek'] = out['datetime'].dt.dayofweek
    out['hour'] = out['datetime'].dt.hour
    return out


def split_by_time(df: pd.DataFrame, train_ratio: float = 0.8, val_ratio: float = 0.1):
    n = len(df)
    n_train = int(n * train_ratio)
    n_val = int(n * val_ratio)
    train = df.iloc[:n_train]
    val = df.iloc[n_train:n_train+n_val]
    test = df.iloc[n_train+n_val:]
    return train, val, test


def build_feature_matrix(df: pd.DataFrame):
    feature_cols = ['temp','atemp','humidity','windspeed','season','holiday','workingday','weather','year','month','dayofweek','hour']
    X = df[feature_cols].to_numpy(dtype=np.float32)
    y = df['count'].astype(np.float32).to_numpy()
    return X, y, feature_cols


df = load_hourly_data()
print('데이터 크기:', df.shape, '기간:', df['datetime'].min(), '→', df['datetime'].max())
df = add_time_features(df)
train_df, val_df, test_df = split_by_time(df)


### 2. 시퀀스 데이터셋 생성 및 LSTM 모델 정의
- 윈도우 크기 24(하루)로 시퀀스 구성, horizon=1
- 단순 LSTM 회귀 헤드


In [ ]:
def make_windows(X: np.ndarray, y: np.ndarray, window: int, horizon: int = 1):
    xs, ys = [], []
    for i in range(len(X) - window - horizon + 1):
        xs.append(X[i:i+window])
        ys.append(y[i+window:i+window+horizon])
    return np.asarray(xs, dtype=np.float32), np.asarray(ys, dtype=np.float32)


class SequenceDataset(Dataset):
    def __init__(self, X_seq: np.ndarray, y_seq: np.ndarray):
        self.X = X_seq
        self.y = y_seq
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


class LSTMRegressor(nn.Module):
    def __init__(self, num_features: int, hidden_size: int = 64, num_layers: int = 2, dropout: float = 0.1):
        super().__init__()
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.head = nn.Linear(hidden_size, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        y = self.head(last)
        return y.squeeze(-1)


### 3. 학습/검증/테스트
- 표준화: train 기준 평균/표준편차
- 학습: Adam + MSE, 베스트 모델 선택
- 예측 vs 실제 시각화


In [ ]:
def train_model(model, train_loader, val_loader, epochs: int, lr: float, device):
    criterion = nn.MSELoss()
    optim = torch.optim.Adam(model.parameters(), lr=lr)
    best_val = float('inf')
    best_state = None
    for ep in range(1, epochs+1):
        model.train()
        tr_sum, n = 0.0, 0
        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            optim.zero_grad()
            loss.backward()
            optim.step()
            tr_sum += float(loss.item()) * len(xb)
            n += len(xb)
        tr_loss = tr_sum / max(n, 1)

        model.eval()
        with torch.no_grad():
            va_sum, n2 = 0.0, 0
            for xb, yb in val_loader:
                xb = xb.to(device)
                yb = yb.to(device)
                pred = model(xb)
                loss = criterion(pred, yb)
                va_sum += float(loss.item()) * len(xb)
                n2 += len(xb)
        va_loss = va_sum / max(n2, 1)
        print(f'Epoch {ep:03d} - train MSE: {tr_loss:.4f}, val MSE: {va_loss:.4f}')
        if va_loss < best_val:
            best_val = va_loss
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
    if best_state is not None:
        model.load_state_dict(best_state)


def predict_all(model, loader, device):
    model.eval()
    outs = []
    with torch.no_grad():
        for xb, _ in loader:
            xb = xb.to(device)
            pred = model(xb)
            outs.append(pred.cpu().numpy())
    return np.concatenate(outs, axis=0)


def plot_series(dt_index, y_true, y_pred, title):
    plt.figure(figsize=(12,4))
    plt.plot(dt_index, y_true, label='Actual')
    plt.plot(dt_index, y_pred, label='Pred')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Count')
    plt.legend()
    plt.tight_layout()
    plt.show()


# 데이터 구성 및 학습
X_train, y_train, feat_cols = build_feature_matrix(train_df)
X_val, y_val, _ = build_feature_matrix(val_df)
X_test, y_test, _ = build_feature_matrix(test_df)

mean = X_train.mean(axis=0, keepdims=True)
std = X_train.std(axis=0, keepdims=True) + 1e-8
X_train = (X_train - mean) / std
X_val = (X_val - mean) / std
X_test = (X_test - mean) / std

window = 24
Xtr_seq, ytr_seq = make_windows(X_train, y_train, window)
Xva_seq, yva_seq = make_windows(X_val, y_val, window)
Xte_seq, yte_seq = make_windows(X_test, y_test, window)

train_loader = DataLoader(SequenceDataset(Xtr_seq, ytr_seq.squeeze(-1) if ytr_seq.ndim>1 else ytr_seq), batch_size=128, shuffle=True)
val_loader = DataLoader(SequenceDataset(Xva_seq, yva_seq.squeeze(-1) if yva_seq.ndim>1 else yva_seq), batch_size=256, shuffle=False)
test_loader = DataLoader(SequenceDataset(Xte_seq, yte_seq.squeeze(-1) if yte_seq.ndim>1 else yte_seq), batch_size=256, shuffle=False)

_set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMRegressor(num_features=Xtr_seq.shape[-1], hidden_size=64, num_layers=2, dropout=0.1).to(device)
train_model(model, train_loader, val_loader, epochs=20, lr=1e-3, device=device)

val_pred = predict_all(model, val_loader, device)
test_pred = predict_all(model, test_loader, device)

val_idx = val_df['datetime'].iloc[window:].values
test_idx = test_df['datetime'].iloc[window:].values
plot_series(val_idx, yva_seq.squeeze(-1), val_pred, 'LSTM 검증 예측 vs 실제')
plot_series(test_idx, yte_seq.squeeze(-1), test_pred, 'LSTM 테스트 예측 vs 실제')


### 4. 성능 평가 및 잔차 분석
- MAE/MSE/RMSE/MAPE, 방향정확도(DA)
- 예측 vs 실제 플롯, 잔차 시각화


In [ ]:
def compute_metrics(y_true, y_pred):
    mae = float(np.mean(np.abs(y_true - y_pred)))
    mse = float(np.mean((y_true - y_pred)**2))
    rmse = float(np.sqrt(mse))
    with np.errstate(divide='ignore', invalid='ignore'):
        mape = float(np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), 1e-8, None))) * 100.0)
    prev = np.concatenate([[y_true[0]], y_true[:-1]])
    da = float(np.mean((np.sign(y_true - prev) == np.sign(y_pred - prev)).astype(float)))
    return mae, mse, rmse, mape, da

print('LSTM/VAL', compute_metrics(yva_seq.squeeze(-1), val_pred))
print('LSTM/TEST', compute_metrics(yte_seq.squeeze(-1), test_pred))

# 잔차 분석
residual = yte_seq.squeeze(-1) - test_pred
plt.figure(figsize=(12,3.5))
plt.plot(test_idx, residual)
plt.title('잔차 시계열 (LSTM)')
plt.tight_layout(); plt.show()

plt.figure(figsize=(6,4))
plt.hist(residual, bins=30)
plt.title('잔차 분포 (LSTM)')
plt.tight_layout(); plt.show()


### 5. 강의 요약 및 다음 단계
- 윈도우 기반 시퀀스 모델의 장단점
- 베이스라인 대비 개선 포인트 및 추가 실험 아이디어(스택드 LSTM, Dropout, 다중 스텝 예측)
